In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
import sqlite3

from traffic_comparator.sqlite import COLUMN_DATATYPES, COLUMN_JSONS, json_load_function, get_took_value, get_latest_table_name

In [ ]:
db_file = Path("comparisons.db")
con = sqlite3.connect(db_file)
cur = con.cursor()
# By default, this reads from the latest table, but this can be modified to a specific table name instead.
table_name = get_latest_table_name(cur)
df = pd.read_sql_query(f"SELECT * from {table_name}", con,
                       dtype=COLUMN_DATATYPES)
con.close()

# This loads the text from each of the `table_json_fields` as a python dictionary
for column in COLUMN_JSONS:
    df[column] = df[column].apply(json_load_function)
    
# This creates the source and target `took` fields by extracting the took value from the response bodies.
df['source_took'] = df['source_response_body'].apply(get_took_value)
df['target_took'] = df['target_response_body'].apply(get_took_value)
df.head()

In [ ]:
df.describe()

In [ ]:
print("Measured Latency")
print(df['target_response_latency'].describe())
print()
print("Took field")
print(df['target_took'].describe())


In [ ]:
# Latency plot
plt.subplot(2, 1, 1)
_, bins, _ = plt.hist([df['source_response_latency'], df['target_response_latency']], bins=60, range=(0, 300), histtype='stepfilled', alpha=0.8)
plt.axvline(df['source_response_latency'].median(), color='k', linestyle='dashed', linewidth=1)
plt.axvline(df['target_response_latency'].median(), color='k', linestyle='dashed', linewidth=1)

plt.title("Measured latency of source vs target cluster")
plt.xlabel("Latency (ms)")
plt.ylabel("Count")

# Took plot
plt.subplot(2, 1, 2)
plt.hist([df['source_took'], df['target_took']], bins=bins, histtype='stepfilled', alpha=0.7)
plt.axvline(df['source_took'].median(), color='k', linestyle='dashed', linewidth=1)
plt.axvline(df['target_took'].median(), color='k', linestyle='dashed', linewidth=1)
plt.title("Reported latency (\"took\") of source vs target cluster")
plt.xlabel("Latency (ms)")
plt.ylabel("Count")
plt.tight_layout()


print(f"Source median latency: {df['source_response_latency'].median():.2f} ms")
print(f"Target median latency: {df['target_response_latency'].median():.2f} ms")
plt.show()
print(f"Source median 'took': {df['source_took'].median():.2f} ms")
print(f"Target median 'took': {df['target_took'].median():.2f} ms")

In [ ]:
# Estimate the network latency
print("Source cluster network latency")
print((df['source_response_latency'] - df['source_took']).mean())
print()
print("Target cluster network latency")
print((df['target_response_latency'] - df['target_took']).mean())

In [ ]:
uri_groups = df.groupby(['request_uri', 'request_method'])
uri_groups.aggregate(func={'responses_are_identical': lambda x: f"{x.mean():.2%}"})

In [ ]:
uri_groups.size()

In [ ]:
bulk_df = df[df["request_uri"] == "/_bulk"]
bulk_df.head()